<h2>Segmenting and Clustering Neighborhoods in Toronto (Part.2)</h2>
Author: Madkhan Kamal


In [1]:
import pandas as pd

</br>
<b>Crawling dataset from wikipedia</b>

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


</br><b>Ignore cells with a borough that is Not assigned</b>


In [3]:
df = df[df.Borough != 'Not assigned']
df.head(6)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned


</br><b>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</b>

In [4]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.loc[df.Neighbourhood == 'Not assigned', 'Borough']
df.head(6)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park


<br><b>Grouping Neighborhood</b>

In [5]:
df = df.groupby(["Postcode","Borough"])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [6]:
df.shape

(103, 3)

</br> <b>Getting lat,long from API</b>
<p>Note: I use geocoder from opencage api, not from google</p>

In [21]:
from opencage.geocoder import OpenCageGeocode

key = '545b37ac0e6149efbc76be6e1df97ecb'
geocoder = OpenCageGeocode(key)

def getCoordinates(postalcode):
    results = geocoder.geocode(postalcode + ', Canada')
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    return lat,long

<br><b> Insert lat,long values to dataframe</b>

In [22]:
df['Latitude'], df['Longitude'] = zip(*df['Postcode'].apply(getCoordinates))


In [23]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",7.042696,125.580595
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",36.879437,10.308030
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.767800,-79.186600
3,M1G,Scarborough,Woburn,43.765717,-79.221898
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900
